# Full Completiontools example
A working sample going over the full process from start to finish

## Importing the package and setting the parameters

In [ ]:
from context import completiontools
import completiontools.utils as utils
import os
import open3d as o3d
tresholdResolution = 0.05 # The max coverage distance
%load_ext autoreload
%autoreload 2

## Getting the Geometries

In [ ]:
ogGeometryPath = os.path.join(os.getcwd(),"localfiles/voxel_grond_pointcloud.ply")
newGeometryPath =  os.path.join(os.getcwd(),"localfiles/GrondSampleMesh.obj")

ogGeometry = completiontools.utils.get_geometry(ogGeometryPath)
newGeometry = completiontools.utils.get_geometry(newGeometryPath)


In [ ]:

utils.show_geometries([ogGeometry, newGeometry])


## Geometry Combination
a function to combine aligned 2 geometries

In [ ]:
combinedGeometry = completiontools.combine_geometry(ogGeometry, newGeometry, tresholdResolution)

In [ ]:
utils.show_geometries([combinedGeometry])

In [ ]:
# Step 1: Create a convex hull of the newGeometry
newGeoHull = completiontools.get_convex_hull(newGeometry)

In [ ]:

utils.show_geometries([utils.get_lineset(newGeoHull), newGeometry])

In [ ]:
# Step 2: Filter out the irrelevant points in the ogGeometry
relevantOg, irrelevantOg = completiontools.get_points_in_hull(ogGeometry, newGeoHull)

In [ ]:
utils.show_geometries([utils.get_lineset(newGeoHull), relevantOg])

In [ ]:
# Step 3: Isolate the not covered points of the ogGeometry compared to the newGeometry
newGeometryPoints = newGeometry.sample_points_poisson_disk(number_of_points=100000)
coveredPoints, unCoveredPoints = utils.filter_pcd_by_distance(relevantOg, newGeometryPoints, tresholdResolution)


In [ ]:
utils.show_geometries([unCoveredPoints, newGeometry])

In [ ]:
# step 4: performm the visibility check
insideList, outsideList = completiontools.check_point_inside_mesh(unCoveredPoints.points, newGeometry)
visiblePoints = o3d.geometry.PointCloud()
visiblePoints.points = o3d.utility.Vector3dVector(outsideList)
invisiblePoints = o3d.geometry.PointCloud()
invisiblePoints.points = o3d.utility.Vector3dVector(insideList)

In [ ]:
utils.show_geometries([visiblePoints, invisiblePoints, newGeometry], True)

In [ ]:
# Step 5: Filter the newGeometryPoints to only keep the changed geometry
existingNewGeo, newNewGeo = utils.filter_pcd_by_distance(newGeometryPoints, relevantOg, tresholdResolution)


In [ ]:
# Step 6: Combine the irrelevant, unchanged and changed geometry
newCombinedGeometry = coveredPoints + invisiblePoints + newNewGeo

In [ ]:
utils.show_geometries([coveredPoints, invisiblePoints, newNewGeo], True)